In [82]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
from matplotlib.pyplot import figure
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn.neighbors import NearestNeighbors
import copy
import easygui
from datetime import datetime
import math
from scipy import stats


CLASSES_PATH = os.path.dirname(os.path.abspath('../Classes/'))
if not (CLASSES_PATH in sys.path):
    sys.path.append(CLASSES_PATH)
from Classes.Files_Handler_Class import Files_Handler
from Classes.CSV_Files_Class import CSV_Files
from Classes.Bcolors_Class import Bcolors as bcolors


In [83]:
colors_list = ['b', 'g', 'r', 'c', 'm', 'y', 'peru', 'tan', 'gold', 'lime', 'teal', 'aqua', 'navy', 'plum', 'pink']

In [84]:
layer_centrality = ['layer_density','layer_degree_histogram','layer_edge_weight',
                    'layer_sombor_index', 'layer_nodes_weight','layer_k_shell_weight']
node_centrality = ['degree', 'clustering', 'nip', 'sombor_index', 'ego_density','ego_degree',
                     'ego_k_shell', 'ego_degree_mean','kss', 'vote_power']
drop_centrality = ['layer_id', 'node_id', 'k_shell', 'k_shell_itr']

In [85]:
def get_content_of_csv_files(path: "str", files_list: "list", drop_culm:list[str]=[]):
    content_list = []
    for item in files_list:
        content_list.append(pd.read_csv(str(path + item)))
        content_list[-1] = content_list[-1].drop(drop_culm, axis=1)
        try:
            content_list[-1] = content_list[-1].drop('weight', axis=1)
        except:
            pass
        null_values = content_list[-1].isnull().sum()
        null_row_cunt = null_values.sum()
        if null_row_cunt > 0:
            print(item, null_row_cunt)
            for j, jtem in enumerate(null_values):
                if jtem > 0 :
                    print(list(content_list[-1])[j], jtem)
        content_list[-1] = content_list[-1].dropna()
    return content_list

def gaussian_kde_func(dataset:pd.DataFrame, clean_data_path:str):
    for column in list(dataset):
        kde = stats.gaussian_kde(dataset[column])
        x = np.linspace(dataset[column].min(), dataset[column].max(), 100)
        y = kde(x)
        plt.figure(figsize=(10,6))
        plt.plot(x, y)
        plt.fill_between(x, y, alpha=0.5)
        plt.title(f'gaussian_kde {column}')
        plt.xlabel(column)
        plt.ylabel('Frequency')
        plt.xlim(math.floor(dataset[column].min()), math.ceil(dataset[column].max()))
        plt.savefig(clean_data_path + f'gaussian_kde {column} before outlier detection.png')
        plt.show()

    pass


In [86]:
def scale_data(data, drop_centrality):
    scale_vector = [10000, 1000000, 1000000000, 100000, 1000000000, 1000000000, 1000000, 100000, 10000, 1, 1, 1, 10, 10, 100, 1, 100]
    scaled_data = copy.deepcopy(data)
    features_list = list(data.keys())
    j = 0
    for i, item in enumerate(features_list):
        if not(item in drop_centrality):
            scaled_data[item] = data[item]/scale_vector[j]
            j += 1
    data = copy.deepcopy(scaled_data)
    del scaled_data
    return data

In [87]:
select_file_object = Files_Handler()
csv_files_object = CSV_Files()
multiple_selecion = False

data_path = select_file_object.select_files("text files", ".csv", 'Select Datasets', multiple_selecion)
path = ""
networks_content = []
if multiple_selecion:
    path = data_path[0][:data_path[0].rfind("/")] + "/"
    for item in data_path:
        networks_content.append(pd.read_csv(item))
else:
    path = data_path[:data_path.rfind("/")] + "/"
    networks_name = select_file_object.get_files_in_path(path)
    print(len(networks_name))
    networks_content = csv_files_object.get_content_of_csv_files(path, networks_name)

# print(networks_content)


42


In [88]:
path
root_path = ('/'.join(path.split('/')[:-2])) + '/'
aggregation_data_path = select_file_object.make_dir(root_path, 'Aggregated_Data')

In [89]:
for i, item in enumerate(networks_content):
    if item.isnull().values.any():
        print(i + 1, networks_name[i])
    

In [90]:
data = pd.concat(networks_content, axis=0)
del networks_content
data = data.sample(frac=1)

In [91]:
print(data.shape)
data.head(5)

(867691, 21)


,layer_id,node_id,degree,k_shell,k_shell_itr,nip,sombor_index,ego_density,ego_degree,ego_k_shell,...,kss,vote_power,clustering,SIR,layer_density,layer_degree_histogram,layer_edge_weight,layer_sombor_index,layer_nodes_weight,layer_k_shell_weight
44960,1,5290,6,3.0,1.0,7048.472896,4247.017525,24.933623,-2400.260463,-36026.374696,...,8.660254,4.0,0.2,2.196,0.000009,2.757014,4.42924,92.682397,32.1307,0.672783
151936,21,97023,2,2.0,1.0,22.000000,10.885661,5.900000,-19.500000,-49.164725,...,5.656854,1.0,0.0,1.037,0.000079,2.487078,0.39072,0.611076,3.1420,1.061614
232964,26,21083,2,2.0,1.0,25.448276,12.718347,4.583333,-15.085786,-319.010921,...,5.656854,1.0,0.0,1.033,0.000066,2.914964,0.64702,0.589333,4.4393,1.486567
174680,14,91168,1,1.0,1.0,39.578680,20.024984,6.850000,-65.500000,-515.180969,...,2.000000,1.0,0.0,1.021,0.000077,2.729173,0.48174,0.732561,3.5303,1.365998
78636,2,366304,1,1.0,1.0,4705.472389,2353.000212,6.772012,-7214.500000,-22147.832143,...,2.000000,1.0,0.0,1.543,0.000015,3.431245,4.01001,34.248138,23.3735,1.216353


In [92]:
data = scale_data(data, drop_centrality)
print(data.shape)
data.head(5)

(867691, 21)


,layer_id,node_id,degree,k_shell,k_shell_itr,nip,sombor_index,ego_density,ego_degree,ego_k_shell,...,kss,vote_power,clustering,SIR,layer_density,layer_degree_histogram,layer_edge_weight,layer_sombor_index,layer_nodes_weight,layer_k_shell_weight
44960,1,5290,0.0006,3.0,1.0,0.007048,4.247018e-06,0.000249,-2.400260e-06,-3.602637e-05,...,0.000087,0.0004,0.2,2.196,0.000009,0.275701,0.442924,0.926824,32.1307,0.006728
151936,21,97023,0.0002,2.0,1.0,0.000022,1.088566e-08,0.000059,-1.950000e-08,-4.916473e-08,...,0.000057,0.0001,0.0,1.037,0.000079,0.248708,0.039072,0.006111,3.1420,0.010616
232964,26,21083,0.0002,2.0,1.0,0.000025,1.271835e-08,0.000046,-1.508579e-08,-3.190109e-07,...,0.000057,0.0001,0.0,1.033,0.000066,0.291496,0.064702,0.005893,4.4393,0.014866
174680,14,91168,0.0001,1.0,1.0,0.000040,2.002498e-08,0.000069,-6.550000e-08,-5.151810e-07,...,0.000020,0.0001,0.0,1.021,0.000077,0.272917,0.048174,0.007326,3.5303,0.013660
78636,2,366304,0.0001,1.0,1.0,0.004705,2.353000e-06,0.000068,-7.214500e-06,-2.214783e-05,...,0.000020,0.0001,0.0,1.543,0.000015,0.343124,0.401001,0.342481,23.3735,0.012164


In [93]:
data.to_csv(aggregation_data_path + "Aggregated_Data.csv", encoding='utf-8', index=False, header=True)